# Deep Hedging in Incomplete Markets — GBM & Heston

**MSc Thesis Experiment Runner**

Runs the full deep hedging pipeline under two market dynamics:
- **GBM** (constant volatility, calibrated to S&P 500)
- **Heston** (stochastic volatility, calibrated to S&P 500)

## Setup
1. **Runtime → Change runtime type → A100 GPU** (Pro+ recommended)
2. Click **Connect**
3. Run **Cell 1** (clone + install)

## Two ways to run
- **Option A (Browser):** Run cells directly in this notebook
- **Option B (VS Code):** Run Cell 2 to get an SSH tunnel, then connect VS Code and use the terminal

In [ ]:
# Cell 1: Clone repo and install dependencies
!git clone https://github.com/thabangTheActuaryCoder/deep-hedging-thesis.git
%cd deep-hedging-thesis
!pip install -q torch numpy matplotlib optuna

import torch
print(f'\nPython: {__import__("sys").version}')
print(f'PyTorch: {torch.__version__}')
print(f'GPU available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'Device: {torch.cuda.get_device_name(0)}')
    mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'Memory: {mem:.1f} GB')

In [ ]:
# Cell 3: Sanity check — all tests should pass
!python -m pytest tests/test_validation.py -v

In [ ]:
# Cell 4 (QUICK TEST): ~10 min on A100, verifies both GBM + Heston pipelines
!python run_experiment.py --quick --market_model both

In [ ]:
# Cell 5 (FULL RUN): both GBM + Heston, 100k paths, ~4-8 hours on A100
# Comment out Cell 4 above before running this
!python run_experiment.py \
    --paths 100000 \
    --N 200 \
    --epochs 1000 \
    --patience 15 \
    --batch_size 2048 \
    --n_trials 60 \
    --seeds 0 1 2 3 4 \
    --substeps 0 5 10 \
    --market_model both

In [ ]:
# Cell 6: Preview GBM validation plots
from IPython.display import Image, display
import glob

print('=== GBM Validation Plots ===')
for img in sorted(glob.glob('outputs/gbm/plots_val/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 7: Preview Heston validation plots
from IPython.display import Image, display
import glob

print('=== Heston Validation Plots ===')
for img in sorted(glob.glob('outputs/heston/plots_val/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 8: Heston stochastic volatility diagnostic plots
from IPython.display import Image, display
import glob

print('=== Heston Stochastic Volatility Diagnostics ===')
for img in sorted(glob.glob('outputs/heston/plots_heston/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 9: GBM vs Heston comparison plots
from IPython.display import Image, display
import glob

print('=== GBM vs Heston Comparison ===')
for img in sorted(glob.glob('outputs/comparison/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 10: 3D delta surface plots
from IPython.display import Image, display
import glob

for label, pattern in [('GBM', 'outputs/gbm/plots_3d/*.png'),
                       ('Heston', 'outputs/heston/plots_3d/*.png')]:
    imgs = sorted(glob.glob(pattern))
    if imgs:
        print(f'\n=== {label} 3D Delta Surfaces ===')
        for img in imgs:
            print(f'\n--- {img} ---')
            display(Image(filename=img, width=700))

In [ ]:
# Cell 11: Show validation metrics (both market models)
import json, os

for market in ['gbm', 'heston']:
    path = f'outputs/{market}/val_metrics.json'
    if not os.path.exists(path):
        continue
    with open(path) as f:
        metrics = json.load(f)
    print(f'\n{"="*50}')
    print(f'  {market.upper()} — Best model: {metrics["best_model"]}')
    print(f'{"="*50}')
    for model, agg in metrics['aggregated_val_metrics'].items():
        cvar = agg['CVaR95_shortfall']
        mse = agg['MSE']
        print(f'  {model:6s}  CVaR95 = {cvar["mean"]:.6f} +/- {cvar["std"]:.6f}  '
              f'MSE = {mse["mean"]:.6f} +/- {mse["std"]:.6f}')

summary_path = 'outputs/metrics_summary.json'
if os.path.exists(summary_path):
    with open(summary_path) as f:
        combined = json.load(f)
    print(f'\n{"="*50}')
    print('  COMBINED SUMMARY')
    print(f'{"="*50}')
    for market, agg in combined.items():
        print(f'\n  [{market.upper()}]')
        for model, m in agg.items():
            cvar = m.get('CVaR95_shortfall', {})
            if isinstance(cvar, dict):
                print(f'    {model:6s}  CVaR95 = {cvar.get("mean",0):.6f} +/- {cvar.get("std",0):.6f}')

In [ ]:
## Replot (optional)

If you want to adjust figure dimensions, colors, grids, or fonts without
rerunning the experiment, edit the `STYLE` dict in `replot.py` and rerun
Cell 13 below. The data was saved during the experiment.

# Cell 13: Regenerate comparison plots from saved data (edit STYLE in replot.py first)
!python replot.py --data outputs/comparison/comparison_data.pt \
                  --metrics outputs/metrics_summary.json \
                  --out outputs/comparison

from IPython.display import Image, display
import glob
for img in sorted(glob.glob('outputs/comparison/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 14: Download all outputs as zip
import shutil
from google.colab import files

shutil.make_archive('outputs', 'zip', '.', 'outputs')
files.download('outputs.zip')

In [ ]:
# Cell 13: Download all outputs as zip
import shutil
from google.colab import files

shutil.make_archive('outputs', 'zip', '.', 'outputs')
files.download('outputs.zip')